In [59]:
import os, datetime, time, re, warnings

from abc import ABCMeta, abstractmethod
from pathlib import Path
from contextlib import contextmanager
import multiprocessing
from multiprocessing import Pool

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from pandas.core.common import SettingWithCopyWarning

from sklearn import datasets, manifold, mixture, model_selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.sparse import vstack, csr_matrix, save_npz, load_npz, hstack

from gensim import corpora, models, similarities
from gensim.models import Word2Vec

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option("display.width", 1000)

In [60]:
@contextmanager
def timer(title):
    t0 = time.time()
    print(f'[{title}] start')
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [61]:
class FeatureEngineering(metaclass=ABCMeta):
    BASE_DIR = "."
    
    def __init__(self):
        self.name = self.__class__.__name__
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.train_file_path = f"{Path(self.BASE_DIR)}/{self.name.lower()}_train"
        self.test_file_path = f"{Path(self.BASE_DIR)}/{self.name.lower()}_test"
    
    @abstractmethod
    def load_data(self):
        raise NotImplementedError

    @abstractmethod
    def create_features(self):
        raise NotImplementedError
        
    def run(self, use_columns=[], skip_columns=[]):
        with timer(self.name):
            self.load_data()
            self.replace_na(use_columns=use_columns, skip_columns=skip_columns)
            self.onehot_encode(use_columns=use_columns, skip_columns=skip_columns)
            self.create_features()
        
        return self
    
    def onehot_encode(self, use_columns=[], skip_columns=[], sparse=False):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        self.label_encode(use_columns, skip_columns)
        if sparse:
            encoder = OneHotEncoder(categories='auto', sparse=sparse, dtype='uint8').fit(pd.concat([self.train.loc[:, use_columns], self.test.loc[:, use_columns]]))
            m = 100000
            train = vstack([encoder.transform(self.train[i*m:(i+1)*m].loc[:, use_columns]) for i in range(self.train.shape[0] // m + 1)])
            test  = vstack([encoder.transform(self.test[i*m:(i+1)*m].loc[:, use_columns])  for i in range(self.test.shape[0] // m +  1)])
            save_npz(f"{self.train_file_path}.npz", train, compressed=True)
            save_npz(f"{self.test_file_path}.npz",  test,  compressed=True)
        else:
            for col in use_columns:
                self.train = self.train.join(pd.get_dummies(self.train[col], prefix=col))
                self.test = self.test.join(pd.get_dummies(self.test[col], prefix=col))
    
        return self
    
    def label_encode(self, use_columns=[], skip_columns=[]):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        for col in use_columns:
            self.train[col] = self.train[col].astype('str')
            self.test[col] = self.test[col].astype('str')
            
            le = LabelEncoder().fit(np.unique(self.train[col].unique().tolist()+self.test[col].unique().tolist()))
            self.train[col] = le.transform(self.train[col])+1
            self.test[col]  = le.transform(self.test[col])+1
    
        return self
    
    def target_encode(self, col_name, target_name, min_samples_leaf=1, smoothing=1, noise_level=0):
        trn_series = self.train[col_name]
        tst_series = self.test[col_name]
        target = self.train[target_name]
        
        assert len(trn_series) == len(target)

        temp = pd.concat([trn_series, target], axis=1)
        averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
        smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
        prior = target.mean()
        averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
        averages.drop(["mean", "count"], axis=1, inplace=True)
        ft_trn_series = pd.merge(
            trn_series.to_frame(trn_series.name),
            averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
            on=trn_series.name,
            how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
        ft_trn_series.index = trn_series.index 
        ft_tst_series = pd.merge(
            tst_series.to_frame(tst_series.name),
            averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
            on=tst_series.name,
            how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
        ft_tst_series.index = tst_series.index

        self.train[f"te_smoothing_{col_name}"], self.test[f"te_smoothing_{col_name}"] = self.__add_noise(ft_trn_series, noise_level), self.__add_noise(ft_tst_series, noise_level)
        
        return self
    
    def agg_transform(self, group, agg, prefix=""):
        if prefix:
            prefix += "_"
        else:
            prefix = f"{'_'.join(group)}_" if type(group) is list else f"{group}_"

        for k, v in agg.items():
            if type(v) is str:
                self.train[f"{prefix}{k}_{v}"] = self.train.groupby(group)[k].transform(v)
                self.test[f"{prefix}{k}_{v}"] = self.test.groupby(group)[k].transform(v)
            else:
                for vv in v:
                    self.train[f"{prefix}{k}_{vv}"] = self.train.groupby(group)[k].transform(vv)
                    self.test[f"{prefix}{k}_{vv}"] = self.test.groupby(group)[k].transform(vv)
        
        return self
    
    def agg_transform_ratio(self, group, agg, prefix=""):
        if prefix:
            prefix += "_"
        else:
            prefix = f"{'_'.join(group)}_" if type(group) is list else f"{group}_"
        prefix = f"ratio_{prefix}"
        
        for k, v in agg.items():
            if type(v) is str:
                self.train[f"{prefix}{k}_{v}"] = self.train[k] / self.train.groupby(group)[k].transform(v)
                self.test[f"{prefix}{k}_{v}"] = self.test[k] / self.test.groupby(group)[k].transform(v)
            else:
                for vv in v:
                    self.train[f"{prefix}{k}_{vv}"] = self.train[k] / self.train.groupby(group)[k].transform(vv)
                    self.test[f"{prefix}{k}_{vv}"] = self.test[k] / self.test.groupby(group)[k].transform(vv)
        
        return self
    
    def replace_na(self, use_columns=[], skip_columns=[], fill_value=-1):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        for col in use_columns:
            if isinstance(self.train[col].dtype, CategoricalDtype):
                self.train[col] = self.train[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(fill_value))
                self.test[col] = self.test[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(fill_value))
            else:
                self.train[col] = self.train[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(fill_value)
                self.test[col] = self.test[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(fill_value)

        return self
    
    def replace_na_mode(self, use_columns=[], skip_columns=[]):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        for col in use_columns:
            self.train[col] = self.train[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.train[col].mode().values[0])
            self.test[col] = self.test[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.test[col].mode().values[0])

        return self
    
    def replace_na_mean(self, use_columns=[], skip_columns=[]):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        for col in use_columns:
            if isinstance(self.train[col].dtype, CategoricalDtype):
                self.train[col] = self.train[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(-1))
                self.test[col] = self.test[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(-1))
            else:
                self.train[col] = self.train[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.train[col].mean())
                self.test[col] = self.test[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.test[col].mean())

        return self

    def replace_na_median(self, use_columns=[], skip_columns=[]):
        use_columns = use_columns if use_columns else [c for c in self.train.columns if c not in skip_columns]
        for col in use_columns:
            if isinstance(self.train[col].dtype, CategoricalDtype):
                self.train[col] = self.train[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(-1))
                self.test[col] = self.test[col].cat.add_categories(str(fill_value)).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(str(-1))
            else:
                self.train[col] = self.train[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.train[col].median())
                self.test[col] = self.test[col].replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(self.test[col].median())

        return self
    
    def calc_topic_score(self, topic_text_columns, num_topics=5):
        df = pd.concat([self.train.loc[:, topic_text_columns], self.test.loc[:, topic_text_columns]])
        
        for col in topic_text_columns:
            texts = [[word for word in document.lower().split()] for document in df[col].values]
            dictionary = corpora.Dictionary(texts)
            bow_corpus = [dictionary.doc2bow(t) for t in texts]
            lda = models.LdaModel(bow_corpus, id2word=dictionary, num_topics=num_topics)
                        
            size = df.shape[0]
            topics = {i:[-1]*size for i in range(num_topics)}
            for i, row in enumerate(lda[bow_corpus]):
                for (topic_num, prop_topic) in row:
                    topics[topic_num][i] = prop_topic
            
            for i in range(num_topics):
                self.train[f"{col}_topic_{i}"] = topics[i][:self.train.shape[0]]
                self.test[f"{col}_topic_{i}"] = topics[i][self.train.shape[0]:]

        return self
    
    def calc_scdv_word2vec_score(self, text_col_name):
        features_num = 20
        min_word_count = 10
        context = 5
        downsampling = 1e-3
        epoch_num = 10
        clusters_num = 6
        
        df = pd.concat([self.train.loc[:, [text_col_name]], self.test.loc[:, [text_col_name]]])
        df[text_col_name] = df[text_col_name].fillna("")
        
        corpus = [self.__analyzer_cat(text) for text in df[text_col_name]]
        word2vecs = Word2Vec(sentences=corpus, iter=epoch_num, size=features_num, min_count=min_word_count, window=context, sample=downsampling)
        word_vectors = word2vecs.wv.vectors
        
        gmm = mixture.GaussianMixture(n_components=clusters_num, covariance_type='tied', max_iter=50)
        gmm.fit(word_vectors)
        
        tfidf_vectorizer = TfidfVectorizer(analyzer=self.__analyzer_cat, min_df=min_word_count)
        tfidfs = tfidf_vectorizer.fit_transform(df[text_col_name])
        
        idf_dic = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer._tfidf.idf_))
        assign_dic = dict(zip(word2vecs.wv.index2word, gmm.predict(word_vectors)))
        soft_assign_dic = dict(zip(word2vecs.wv.index2word, gmm.predict_proba(word_vectors)))
        
        word_topic_vecs = {}
        for word in assign_dic:
            word_topic_vecs[word] = np.zeros(features_num*clusters_num, dtype=np.float32)
            for i in range(0, clusters_num):
                try:
                    word_topic_vecs[word][i*features_num:(i+1)*features_num] = word2vecs.wv[word]*soft_assign_dic[word][i]*idf_dic[word]
                except:
                    continue
        
        scdvs = np.zeros((len(df[text_col_name]), clusters_num*features_num), dtype=np.float32)

        a_min = 0
        a_max = 0

        for i, text in enumerate(df[text_col_name]):
            tmp = np.zeros(clusters_num*features_num, dtype=np.float32)
            words = self.__analyzer_cat(text)
            for word in words:
                if word in word_topic_vecs:
                    tmp += word_topic_vecs[word]
            norm = np.sqrt(np.sum(tmp**2))
            if norm > 0:
                tmp /= norm
            a_min += min(tmp)
            a_max += max(tmp)
            scdvs[i] = tmp

        p = 0.04
        a_min = a_min*1.0 / len(df[text_col_name])
        a_max = a_max*1.0 / len(df[text_col_name])
        thres = (abs(a_min)+abs(a_max)) / 2
        thres *= p
        scdvs[abs(scdvs) < thres] = 0
        
        tsne_scdv = manifold.TSNE(n_components=2).fit_transform(scdvs)
        
        self.train[f"scdv_{text_col_name}_x"] = tsne_scdv[:self.train.shape[0], 0]
        self.train[f"scdv_{text_col_name}_y"] = tsne_scdv[:self.train.shape[0], 1]        
        self.test[f"scdv_{text_col_name}_x"] = tsne_scdv[self.train.shape[0]:, 0]
        self.test[f"scdv_{text_col_name}_y"] = tsne_scdv[self.train.shape[0]:, 1]
        
        return self
    
    def columns_1d(self):
        self.train.columns = pd.Index([(e[0] + "_" + e[1].lower()) if (len(e[1]) > 0) else e[0] for e in self.train.columns.tolist()])
        self.test.columns = pd.Index([(e[0] + "_" + e[1].lower()) if (len(e[1]) > 0) else e[0] for e in self.test.columns.tolist()])

        return self
    
    def head(self, title="", columns=[], limit=5):
        train_cols, test_cols = (columns, columns) if columns else (self.train.columns, self.test.columns)
        
        print(f"train head: {title}")
        print(self.train.loc[:, train_cols].head(limit))
        print("----------------------------")
        print(f"test head: {title}")
        print(self.test.loc[:, test_cols].head(limit))
        print("----------------------------")
        
        return self
    
    def tail(self, title="", columns=[], limit=5):
        train_cols, test_cols = (columns, columns) if columns else (self.train.columns, self.test.columns)
        
        print(f"train tail: {title}")
        print(self.train.loc[:, train_cols].tail(limit))
        print("----------------------------")
        print(f"test tail: {title}")
        print(self.test.loc[:, test_cols].tail(limit))
        print("----------------------------")
        
        return self
    
    def save(self, format="feather", index=False):
        if format == "feather":
            self.train.to_feather(f"{self.train_file_path}.ftr")
            self.test.to_feather(f"{self.test_file_path}.ftr")
        elif format == "csv":
            self.train.to_csv(f"{self.train_file_path}.csv", index=index)
            self.test.to_csv(f"{self.test_file_path}.csv", index=index)
        
        return self
    
    def __add_noise(self, series, noise_level):
        return series * (1 + noise_level * np.random.randn(len(series)))

    def __analyzer_nlp(self, text):
        stop_words = ['i', 'a', 'an', 'the', 'to', 'and', 'or', 'if', 'is', 'are', 'am', 'it', 'this', 'that', 'of', 'from', 'in', 'on']
        text = text.lower()
        text = text.replace('\n', '')
        text = text.replace('\t', '')
        text = re.sub(re.compile(r'[!-\/:-@[-`{-~]'), ' ', text)
        text = text.split(' ')

        words = []
        for word in text:
            if (re.compile(r'^.*[0-9]+.*$').fullmatch(word) is not None):
                continue
            if word in stop_words:
                continue
            if len(word) < 2:
                continue
            words.append(word)

        return words

    def __analyzer_cat(self, text):
        return text.split(' ')

In [62]:
class Sample(FeatureEngineering):
    def load_data(self):
        self.train = pd.read_csv("../sample-data/train.csv", nrows=100)
        self.test = pd.read_csv("../sample-data/test.csv", nrows=100)
        
        return self
    
    def create_features(self):
        return self
    
    def create_topic_text(self):
        self.train["count_1"] = self.train["first_active_month"].astype("str") + self.train.groupby("first_active_month")["first_active_month"].transform("count").astype("int").astype("str")
        self.train["count_2"] = self.train["feature_1"].astype("int").astype("str") + self.train.groupby("feature_1")["feature_1"].transform("count").astype("int").astype("str")
        self.train["topic_text"] = "A"+self.train["count_1"].astype(str) \
                            +" B"+self.train["count_2"].astype(str)
        
        self.test["count_1"] = self.test["first_active_month"].astype("str") + self.test.groupby("first_active_month")["first_active_month"].transform("count").astype("int").astype("str")
        self.test["count_2"] = self.test["feature_1"].astype("int").astype("str") + self.test.groupby("feature_1")["feature_1"].transform("count").astype("int").astype("str")
        self.test["topic_text"] = "A"+self.test["count_1"].astype(str) \
                            +" B"+self.test["count_2"].astype(str)
        
        return self

In [63]:
s = Sample()
# s.load_data().head(title="before label_encode").label_encode(skip_columns=["target"]).head(title="after label_encode", limit=10).agg_transform(group=["feature_1"], agg={"first_active_month": ["min", "max", "mean"]}).tail()
# s.run(skip_columns=["target"]).agg_transform(group="first_active_month", agg={"feature_1_1": ["min", "max", "mean"]}).head(columns=["card_id", "feature_1_1_max"]).save()
s.load_data().head()

train head: 
  first_active_month          card_id  feature_1  feature_2  feature_3    target
0            2017-06  C_ID_92a2005557          5          2          1 -0.820283
1            2017-01  C_ID_3d0044924f          4          1          0  0.392913
2            2016-08  C_ID_d639edf6cd          2          2          0  0.688056
3            2017-09  C_ID_186d6a6901          4          3          0  0.142495
4            2017-11  C_ID_cdbd2c0db2          1          3          0 -0.159749
----------------------------
test head: 
  first_active_month          card_id  feature_1  feature_2  feature_3
0            2017-04  C_ID_0ab67a22ab          3          3          1
1            2017-01  C_ID_130fd0cbdd          2          3          0
2            2017-08  C_ID_b709037bc5          5          1          1
3            2017-12  C_ID_d27d835a9f          2          1          0
4            2015-12  C_ID_2b5e3df5c2          5          1          1
----------------------------


In [64]:
s.target_encode(col_name="feature_1", target_name="target", min_samples_leaf=100, smoothing=10, noise_level=0.01).head()

train head: 
  first_active_month          card_id  feature_1  feature_2  feature_3    target  te_smoothing_feature_1
0            2017-06  C_ID_92a2005557          5          2          1 -0.820283               -0.703912
1            2017-01  C_ID_3d0044924f          4          1          0  0.392913               -0.713972
2            2016-08  C_ID_d639edf6cd          2          2          0  0.688056               -0.712038
3            2017-09  C_ID_186d6a6901          4          3          0  0.142495               -0.725263
4            2017-11  C_ID_cdbd2c0db2          1          3          0 -0.159749               -0.724358
----------------------------
test head: 
  first_active_month          card_id  feature_1  feature_2  feature_3  te_smoothing_feature_1
0            2017-04  C_ID_0ab67a22ab          3          3          1               -0.714440
1            2017-01  C_ID_130fd0cbdd          2          3          0               -0.708047
2            2017-08  C_ID_b709

In [65]:
s.load_data().create_topic_text().head()

train head: 
  first_active_month          card_id  feature_1  feature_2  feature_3    target    count_1 count_2       topic_text
0            2017-06  C_ID_92a2005557          5          2          1 -0.820283   2017-067     520   A2017-067 B520
1            2017-01  C_ID_3d0044924f          4          1          0  0.392913   2017-019     413   A2017-019 B413
2            2016-08  C_ID_d639edf6cd          2          2          0  0.688056   2016-086     235   A2016-086 B235
3            2017-09  C_ID_186d6a6901          4          3          0  0.142495  2017-0913     413  A2017-0913 B413
4            2017-11  C_ID_cdbd2c0db2          1          3          0 -0.159749   2017-116      14    A2017-116 B14
----------------------------
test head: 
  first_active_month          card_id  feature_1  feature_2  feature_3   count_1 count_2      topic_text
0            2017-04  C_ID_0ab67a22ab          3          3          1  2017-048     336  A2017-048 B336
1            2017-01  C_ID_130fd0c

In [66]:
s.calc_scdv_word2vec_score("topic_text").head()

train head: 
  first_active_month          card_id  feature_1  feature_2  feature_3    target    count_1 count_2       topic_text  scdv_topic_text_x  scdv_topic_text_y
0            2017-06  C_ID_92a2005557          5          2          1 -0.820283   2017-067     520   A2017-067 B520          -0.025154          -6.124799
1            2017-01  C_ID_3d0044924f          4          1          0  0.392913   2017-019     413   A2017-019 B413          -0.701709           1.323653
2            2016-08  C_ID_d639edf6cd          2          2          0  0.688056   2016-086     235   A2016-086 B235           6.286077         -10.981548
3            2017-09  C_ID_186d6a6901          4          3          0  0.142495  2017-0913     413  A2017-0913 B413          -1.336526           0.664063
4            2017-11  C_ID_cdbd2c0db2          1          3          0 -0.159749   2017-116      14    A2017-116 B14          -1.284992          -2.359389
----------------------------
test head: 
  first_active_m

In [67]:
s.train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,count_1,count_2,topic_text,scdv_topic_text_x,scdv_topic_text_y
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,2017-067,520,A2017-067 B520,-0.025154,-6.124799
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,2017-019,413,A2017-019 B413,-0.701709,1.323653
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,2016-086,235,A2016-086 B235,6.286077,-10.981548
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,2017-0913,413,A2017-0913 B413,-1.336526,0.664063
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017-116,14,A2017-116 B14,-1.284992,-2.359389


In [27]:
s.test.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,count_18,count_19,count_20,count_21,topic_text,topic_text_topic_0,topic_text_topic_1,topic_text_topic_2,topic_text_topic_3,topic_text_topic_4
0,0000010489e3af074adeac69c53e555e,win8defender,1.1.15400.5,4.18.1810.5,1.281.501.0,0,7.0,0,-1.0,53447,...,8071,4.18.1810.531,5344764,-135,A798 B5344764 C585521 D1814 E15063.0.amd64fre....,-1.0,-1.0,-1.000000,-1.000000,0.963180
1,00000176ac758d54827acd545b6315a5,win8defender,1.1.15400.4,4.18.1809.2,1.279.301.0,0,7.0,0,-1.0,53447,...,5549,4.18.1809.238,5344764,RequireAdmin23,A798 B5344764 C713951 D-138 E16299.431.amd64fr...,-1.0,-1.0,0.962994,-1.000000,-1.000000
2,0000019dcefc128c2d4387c1273dae1d,win8defender,1.1.15300.6,4.18.1809.2,1.277.230.0,0,7.0,0,-1.0,49480,...,55615,4.18.1809.238,494802,RequireAdmin16,A798 B494802 C662023 D-138 E14393.2189.amd64fr...,-1.0,-1.0,-1.000000,-1.000000,0.963127
3,0000055553dc51b1295785415f1a224d,win8defender,1.1.15400.5,4.18.1810.5,1.281.664.0,0,7.0,0,-1.0,42160,...,62820,4.18.1810.531,421601,RequireAdmin16,A798 B421601 C1209171 D-138 E16299.15.amd64fre...,-1.0,-1.0,-1.000000,0.846206,0.126227
4,00000574cefffeca83ec8adf9285b2bf,win8defender,1.1.15400.4,4.18.1809.2,1.279.236.0,0,7.0,0,-1.0,53447,...,55615,4.18.1809.238,5344764,RequireAdmin23,A798 B5344764 C1247361 D1814 E16299.15.amd64fr...,-1.0,-1.0,-1.000000,-1.000000,0.963245
